In [24]:
import pandas as pd
import gzip 
import IPython.display
import matplotlib.pyplot as plt
import glob

In [ ]:
from IPython.display import display, HTML
display(HTML('<style>.container {width: 100%} !i'))

In [ ]:
columns = ["date", "time", "status", "latency", "r_bytes", "method", "uri"]

In [ ]:
columns_to_use = ["date","time","status"]
for file in files:
    df = pd.read_csv(file, compression='gzip', header=None, names=columns, skiprows=1, sep=',', quotechar='"', engine='python', usecols=columns_to_use,
         parse_dates={'datetime': ['date', 'time']}, dtype={'status': 'int16', 'latency': 'float32', 'r_bytes': 'int16'})
    #df['datetime'] = pd.to_datetime(df['datetime'])
    #df = df.set_index('datetime')
    dfs.append(df)
print("gzip load done ...")
df = pd.concat(dfs)
print("came together")

In [28]:
mask = df['uri'].str.contains('login')
indices = df[mask].index
df.drop(indices, inplace=True)


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 10)

In [ ]:
#df.head(10)

In [ ]:
pd.get_option("display.max_columns")
pd.get_option("display.max_rows")

In [ ]:
'''
subpath = df['uri'].str.split('/').str[:2].str.join('/')
print(subpath)
df['subpath'] = subpath
'''

In [ ]:
# max_r_bytes = df['r_bytes'].max()
# print("Max r_bytes:", max_r_bytes)

In [ ]:
# latency_stats = latency_stats.apply(lambda x: format(x, '.2f'))
# print(latency_stats)

latency_stats = df['latency'].describe()
print(latency_stats)

In [ ]:
latency_median = df['latency'].median()
latency_95th_percentile = df['latency'].quantile(0.95)
print(latency_95th_percentile)

In [ ]:
subpath_avg_latency = df.groupby('subpath')['latency'].mean()
sorted_subpaths = subpath_avg_latency.sort_values(ascending=False)
top_5_subpaths = sorted_subpaths.head(5)
print(top_5_subpaths)

In [ ]:
highest_latencies = df.nlargest(5, 'latency')
print(highest_latencies)


In [ ]:
status_counts = df.groupby('status').size().to_frame('count')
status_counts['percentage'] = status_counts['count'] / df.shape[0] * 100
status_counts.reset_index(inplace=True)
print(status_counts[status_counts['status'].isin([502, 500, 499])])


In [ ]:
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

In [ ]:
import matplotlib.pyplot as plt

# Filter out rows where status is 502 and resample to get counts per hour
error_502_counts = df[df['status'] == 502].resample('H').size()

# Filter out rows where status is 500 and resample to get counts per hour
error_500_counts = df[df['status'] == 500].resample('H').size()

# Filter out rows where status is 499 and resample to get counts per hour
error_499_counts = df[df['status'] == 499].resample('H').size()

error_502_counts = error_502_counts.fillna(0)
error_500_counts = error_500_counts.fillna(0)
error_499_counts = error_499_counts.fillna(0)

# Plot the time series of error counts
error_502_counts.plot(label='502', legend=True)
error_500_counts.plot(label='500', legend=True)

# Check if there are any '499' errors before attempting to plot
if not error_499_counts.empty:
    error_499_counts.plot(label='499', legend=True)

plt.title('Time Series of Hourly Error Counts')
plt.xlabel('Date and Hour')
plt.ylabel('Count')
plt.show()
